In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import math
import os
import sys
import csv
import time

data = pd.read_csv("data_OptiMinds.csv")  
# Convert fractional years to datetime to understand the data better
def fractional_year_to_date(year):
    year_int = int(year)
    remainder = year - year_int
    start_of_year = datetime.datetime(year_int, 1, 1)
    days_in_year = (datetime.datetime(year_int + 1, 1, 1) - start_of_year).days
    return start_of_year + datetime.timedelta(days=remainder * days_in_year)

# Apply conversion
data['Date'] = data['Time'].apply(fractional_year_to_date)

subset_random = data.sample(n=1000, random_state=42)
print(subset_random.head())

              Time          SN                       Date
3318   1764.202980   61.059679 1764-03-15 06:58:50.319951
32638  1845.646723   43.003447 1845-08-25 01:17:44.775877
27227  1830.616297   82.596295 1830-08-13 22:45:48.499199
24615  1823.360804    0.000000 1823-05-12 16:38:39.795686
29045  1835.666254  132.597282 1835-09-01 04:22:56.440623


In [2]:
def solar_cycle_model(t, params, num_cycles=10):
    if len(params) != 3 * num_cycles:
        raise ValueError(f"Expected {3 * num_cycles} parameters, got {len(params)}.")
    # Rest of the function...
    predicted_values = np.zeros_like(t)

    for k in range(num_cycles):
        T0 = params[3 * k]       # Start time of cycle k
        Ts = params[3 * k + 1]   # Rising time of cycle k
        Td = params[3 * k + 2]   # Declining time of cycle k

        # Compute the contribution of cycle k
        x_k = ((t - T0) / Ts) ** 2 * np.exp(-((t - T0) / Td) ** 2)

        # Set contributions to zero outside the valid range
        x_k[(t < T0) | (t > T0 + Ts + Td)] = 0
        predicted_values += x_k

    return predicted_values


In [3]:
# Loss Function: Mean Squared Error
def mse(params, t, observed_values):
    """
    Computes the Mean Squared Error (MSE) between observed data and model predictions.
    
    Args:
        params (ndarray): Model parameters [T0_1, Ts_1, Td_1, ..., T0_n, Ts_n, Td_n].
        t (ndarray): Time points of the observations.
        observed_values (ndarray): Observed sunspot data.
    
    Returns:
        float: The MSE value.
    """
    if t.shape != observed_values.shape:
        raise ValueError("Time points (t) and observed values must have the same shape.")

    predicted_values = solar_cycle_model(t, params)
    return np.mean((observed_values - predicted_values) ** 2)

In [4]:
import numpy as np

def simulated_annealing(x0, T0, sigma, f, n_iter=250000, burn_in=200000, verbose=True):
    """
    Performs Simulated Annealing to optimize a given function.
    
    Args:
        x0 (ndarray): Initial parameter guess.
        T0 (float): Initial temperature.
        sigma (float): Standard deviation for the proposal distribution.
        f (function): Function to minimize.
        n_iter (int): Total number of iterations (default is 250,000).
        burn_in (int): Burn-in period (default is 200,000).
        verbose (bool): Whether to print progress updates (default is True).
    
    Returns:
        ndarray: Array of sampled parameter values after burn-in.
    """
    # Validate inputs
    if burn_in >= n_iter:
        raise ValueError("Burn-in period must be less than total number of iterations.")
    if len(x0) == 0:
        raise ValueError("Initial parameter guess x0 cannot be empty.")
    
    # Initialize variables
    x = np.array(x0, dtype=float)  # Ensure x0 is a NumPy array
    T = T0  # Initial temperature
    n_params = x.shape[0]  # Number of parameters to optimize
    prev_loss = f(x)  # Initial loss
    size_out = int(n_iter - burn_in)  # Number of samples after burn-in
    v = np.zeros((size_out, n_params))  # Array to store results after burn-in

    # Precompute constant covariance matrix for proposal distribution
    means = np.zeros(n_params)
    cov_matrix = np.diag(np.full(n_params, sigma))
    
    iter_counter = 0
    accepted = 0

    if verbose:
        print(f"Initial loss: {prev_loss:.6f}")

    # Main optimization loop
    while iter_counter < n_iter:
        iter_counter += 1
        x_old = x.copy()
        
        # Generate a new proposal
        x_proposal = x_old + np.random.multivariate_normal(means, cov_matrix)
        
        # Compute loss difference
        loss_old = f(x_old)
        loss_proposal = f(x_proposal)
        DeltaE = loss_proposal - loss_old

        # Metropolis acceptance step
        if DeltaE < 0 or np.exp(-np.clip(DeltaE / T, -100, 100)) >= np.random.rand():
            x = x_proposal
            accepted += 1
        
        # Update temperature using linear schedule
        T = T0 * (1 - iter_counter / n_iter)

        # Check for early stopping based on loss improvement
        if iter_counter > burn_in and np.abs(loss_old - loss_proposal) < 1e-6:
            if verbose:
                print("Early stopping: Convergence achieved.")
            break

        # Store parameters after burn-in
        if iter_counter > burn_in:
            v[iter_counter - burn_in - 1, :] = x

        # Periodic progress update
        if verbose and iter_counter % 10000 == 0:
            print(f"Iteration {iter_counter}/{n_iter}, "
                  f"Temperature: {T:.6f}, "
                  f"Loss: {f(x):.6f}, "
                  f"Acceptance Rate: {accepted / iter_counter:.4f}")

    if verbose:
        print("Optimization complete.")
    return v
